## The Traveling Salesman Problem

The traveling salesman problem (TSP) sets out to find the *shortest possible route* that a traveling salesman can take to visit a set of cities and return to the origin city.

We are given $n$ cities, and the distance between each pair of cities is defined in a distance matrix $D$, where $D_{ij}$ represents the distance from city $i$ to city $j$. The objective is to **determine the sequence in which the cities should be visited to minimize the total distance traveled.** The sequence must start and end at the same city, ensuring that each city is visited exactly once.

To formulate the TSP in terms of linear programming, binary decision variables $x_{ij}$ are introduced. These variables take the value $1$ if the route from city $i$ to city $j$ is included in the tour, and $0$ otherwise. The objective function then becomes the minimization of the total travel distsance, which is the sum of the distances for each of the paths included in the route:

$$\sum^n_{i=1} \sum^n_{j=1,j \neq i} D_{ij}x_{ij}$$

The constraints of the TSP ensure that each city is part of the route and that the route forms a continuous path without any reptitions or subtours. Specifically, for every city $i$, there must be **exactly one outgoing path and one incoming path in the solution**. This is represented mathematically as

$$\sum^n_{j=1,j \neq i} x_{ij} = 1$$

for every $i$ and 

$$\sum^n_{j=1,j \neq i}x_{ji} = 1$$

for every $i$, ensuring that the route enters and leaves each city $i$ exactly once.

These constraints alone are insufficient to prevent subtours, which are smaller loops within the route that do not cover all cities. To address this, we introduce subtour elimination. A common approach involves using auxiliary variables, $u_i$ for each city $i$ to represent the order of the cities in the tour. The Miller-Tucker-Zemlin (MTZ) constraints are then applied: for each pair of cities $i$ and $j$:

$$u_{i} - u_{j} + nx_{ij} \le n - 1$$

provided $i \neq j$ and excluding the starting city. These constraints effectively prevent the formation of subtours by enforcing an order in the sequence of cities visited.

## Solution

In [139]:
import pandas as pd
from pulp import *

dist = pd.read_csv("dist.csv", index_col=0).values
dist

array([[ 0, 29, 82, 46, 68, 52, 72, 42, 51, 55, 29, 74, 23, 72, 46],
       [29,  0, 55, 46, 42, 43, 43, 23, 23, 31, 41, 51, 11, 52, 21],
       [82, 55,  0, 68, 46, 55, 23, 43, 41, 29, 79, 21, 64, 31, 51],
       [46, 46, 68,  0, 82, 15, 72, 31, 62, 42, 21, 51, 51, 43, 64],
       [68, 42, 46, 82,  0, 74, 23, 52, 21, 46, 82, 58, 46, 65, 23],
       [52, 43, 55, 15, 74,  0, 61, 23, 55, 31, 33, 37, 51, 29, 59],
       [72, 43, 23, 72, 23, 61,  0, 42, 23, 31, 77, 37, 51, 46, 33],
       [42, 23, 43, 31, 52, 23, 42,  0, 33, 15, 37, 33, 33, 31, 37],
       [51, 23, 41, 62, 21, 55, 23, 33,  0, 29, 62, 46, 29, 51, 11],
       [55, 31, 29, 42, 46, 31, 31, 15, 29,  0, 51, 21, 41, 23, 37],
       [29, 41, 79, 21, 82, 33, 77, 37, 62, 51,  0, 65, 42, 59, 61],
       [74, 51, 21, 51, 58, 37, 37, 33, 46, 21, 65,  0, 61, 11, 55],
       [23, 11, 64, 51, 46, 51, 51, 33, 29, 41, 42, 61,  0, 62, 23],
       [72, 52, 31, 43, 65, 29, 46, 31, 51, 23, 59, 11, 62,  0, 59],
       [46, 21, 51, 64, 23, 59, 33

In [194]:
n = 15

tsp = LpProblem("TSP", LpMinimize)

x = LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(n)), cat="Binary")

tsp += lpSum([dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j])

for i in range(n):
    tsp += lpSum(x[i, j] for j in range(n) if i != j) == 1
    tsp += lpSum(x[j, i] for j in range(n) if i != j) == 1

u = LpVariable.dicts("u", range(n), 0, n - 1, "Continuous")

for i in range(1, n):
    for j in range(1, n):
        if i != j:
            tsp += u[i] - u[j] + n * x[i, j] <= n - 1

tsp.solve()

if tsp.status == 1:
    route = []
    for i in range(n):
        for j in range(n):
            if x[i, j].varValue == 1:
                route.append((i, j))

for i in range(len(route) - 1):
    for j in range(i + 1, len(route)):
        if route[i][1] == route[j][0]:
            route[i + 1], route[j] = route[j], route[i + 1]

route, sum([dist[r] for r in route]) == tsp.objective.value()

([(0, 12),
  (12, 1),
  (1, 14),
  (14, 8),
  (8, 4),
  (4, 6),
  (6, 2),
  (2, 11),
  (11, 13),
  (13, 9),
  (9, 7),
  (7, 5),
  (5, 3),
  (3, 10),
  (10, 0)],
 True)